# Amplitude model with `sympy`

In this notebook, we formulate the amplitude model for the $\gamma p \to K^+ \pi^0 \Lambda $ symbolically by adapting the model originally for the $\gamma p \to \eta\pi^0 p$ channel example as described in [Reaction and Models](reaction-model.md).

The model we want to implement is

$$
\begin{array}{rcl}
I &=& \left|A^{12} + A^{23} + A^{31}\right|^2 \\
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s_{12}-m^2_{K^{*+}_2}+im_{K^{*+}_2} \Gamma_{K^{*+}_2}} \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s_{23}-m^2_{\Sigma^*}+im_{\Sigma^*} \Gamma_{\Sigma^*}} \\
A^{31} &=& \frac{c_0}{s_{31}-m^2_{N^{*+}}+im_{N^{*+}} \Gamma_{N^{*+}}} \,,
\end{array}
$$

where $1=K^+$, $2=\pi^0$, and $3=\Lambda$.

In [ ]:
from __future__ import annotations

import logging
import os
import warnings

import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.kinematics.angles import Phi, Theta
from ampform.kinematics.lorentz import (
    ArrayMultiplication,
    ArraySize,
    BoostZMatrix,
    Energy,
    EuclideanNorm,
    FourMomentumSymbol,
    RotationYMatrix,
    RotationZMatrix,
    ThreeMomentum,
    three_momentum_norm,
)
from ampform.sympy import unevaluated
from ampform.sympy._array_expressions import ArraySum
from IPython.display import Latex
from tensorwaves.data import (
    SympyDataTransformer,
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
)

STATIC_PAGE = "EXECUTE_NB" in os.environ

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)
warnings.filterwarnings("ignore")

## Model implementation

In [ ]:
l_max = 2

### $A^{12}$

In [ ]:
s12, m_Kstar2, Gamma_Kstar2, l12 = sp.symbols(r"s_{12} m_{K^*_2} \Gamma_{K^*_2} l_{12}")
theta1, phi1 = sp.symbols("theta_1 phi_1")
a = sp.IndexedBase("a")
m = sp.symbols("m", cls=sp.Idx)
A12 = sp.Sum(a[m] * sp.Ynm(l12, m, theta1, phi1), (m, -l12, l12)) / (
    s12 - m_Kstar2**2 + sp.I * m_Kstar2 * Gamma_Kstar2
)
A12

In [ ]:
A12_funcs = [
    sp.lambdify(
        [
            s12,
            *(a[j] for j in range(-l_max, l_max + 1)),
            m_Kstar2,
            Gamma_Kstar2,
            theta1,
            phi1,
        ],
        expr=A12.subs(l12, i).doit().expand(func=True),
    )
    for i in range(l_max + 1)
]
A12_funcs

### $A^{23}$

In [ ]:
s23, m_Sigma, Gamma_Sigma, l23 = sp.symbols(
    r"s_{23} m_{\Sigma^{*+}} \Gamma_{\Sigma^{*+}} l_{23}"
)
b = sp.IndexedBase("b")
m = sp.symbols("m", cls=sp.Idx)
theta2, phi2 = sp.symbols("theta_2 phi_2")
A23 = sp.Sum(b[m] * sp.Ynm(l23, m, theta2, phi2), (m, -l23, l23)) / (
    s23 - m_Sigma**2 + sp.I * m_Sigma * Gamma_Sigma
)
A23

In [ ]:
A23_funcs = [
    sp.lambdify(
        [
            s23,
            *(b[j] for j in range(-l_max, l_max + 1)),
            m_Sigma,
            Gamma_Sigma,
            theta2,
            phi2,
        ],
        A23.subs(l23, i).doit().expand(func=True),
    )
    for i in range(l_max + 1)
]
A23_funcs

### $A^{31}$

In [ ]:
s31, m_Nstar, Gamma_Nstar = sp.symbols(r"s_{31} m_{N^*} \Gamma_{N^*}")
c = sp.IndexedBase("c")
theta3, phi3, l31 = sp.symbols("theta_3 phi_3 l_{31}")
A31 = sp.Sum(c[m] * sp.Ynm(l31, m, theta3, phi3), (m, -l31, l31)) / (
    s31 - m_Nstar**2 + sp.I * m_Nstar * Gamma_Nstar
)
A31

In [ ]:
A31_funcs = [
    sp.lambdify(
        [
            s31,
            *(c[j] for j in range(-l_max, l_max + 1)),
            m_Nstar,
            Gamma_Nstar,
            theta3,
            phi3,
        ],
        A31.subs(l31, i).doit().expand(func=True),
    )
    for i in range(l_max + 1)
]
A31_funcs

### $I = |A|^2 = |A^{12}+A^{23}+A^{31}|^2$

In [ ]:
intensity_expr = sp.Abs(A12 + A23 + A31) ** 2
intensity_expr

### Phase Space Generation

Mass for $p\gamma$ system

In [ ]:
E_lab_gamma = 8.5
m_proton = 0.938
m_0 = np.sqrt(2 * E_lab_gamma * m_proton + m_proton**2)
m_lambda = 1.12
m_k = 0.494
m_pi = 0.135
m_0

In [ ]:
rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=m_0,
    final_state_masses={1: m_k, 2: m_pi, 3: m_lambda},
)
phsp_momenta = phsp_generator.generate(500_000, rng)

### Kinematic variables

In [ ]:
@unevaluated
class SquaredInvariantMass(sp.Expr):
    momentum: sp.Basic
    _latex_repr_ = "m_{{{momentum}}}^2"

    def evaluate(self) -> sp.Expr:
        p = self.momentum
        p_xyz = ThreeMomentum(p)
        return Energy(p) ** 2 - EuclideanNorm(p_xyz) ** 2


def formulate_helicity_angles(
    pi: FourMomentumSymbol, pj: FourMomentumSymbol
) -> tuple[Theta, Phi]:
    pij = ArraySum(pi, pj)
    beta = three_momentum_norm(pij) / Energy(pij)
    Rz = RotationZMatrix(-Phi(pij), n_events=ArraySize(Phi(pij)))
    Ry = RotationYMatrix(-Theta(pij), n_events=ArraySize(Theta(pij)))
    Bz = BoostZMatrix(beta, n_events=ArraySize(beta))
    pi_boosted = ArrayMultiplication(Bz, Ry, Rz, pi)
    return Theta(pi_boosted), Phi(pi_boosted)

In [ ]:
p1 = FourMomentumSymbol("p1", shape=[])
p2 = FourMomentumSymbol("p2", shape=[])
p3 = FourMomentumSymbol("p3", shape=[])
p12 = ArraySum(p1, p2)
p23 = ArraySum(p2, p3)
p31 = ArraySum(p3, p1)

theta1_expr, phi1_expr = formulate_helicity_angles(p1, p2)
theta2_expr, phi2_expr = formulate_helicity_angles(p2, p3)
theta3_expr, phi3_expr = formulate_helicity_angles(p3, p1)

s12_expr = SquaredInvariantMass(p12)
s23_expr = SquaredInvariantMass(p23)
s31_expr = SquaredInvariantMass(p31)

In [ ]:
kinematic_variables = {
    theta1: theta1_expr,
    theta2: theta2_expr,
    theta3: theta3_expr,
    phi1: phi1_expr,
    phi2: phi2_expr,
    phi3: phi3_expr,
    s12: s12_expr,
    s23: s23_expr,
    s31: s31_expr,
}

Latex(aslatex(kinematic_variables))

In [ ]:
helicity_transformer = SympyDataTransformer.from_sympy(
    kinematic_variables, backend="jax"
)

In [ ]:
phsp = helicity_transformer(phsp_momenta)
list(phsp)

### Parameters

In [ ]:
a_vals = [0, 1.0, 3.0, 3.5, 2.0]  # Slight adjustment to emphasize higher waves
b_vals = [0, -1.0, 3.5, 0.5, 0]  # Adjust for new final state coupling
c_vals = [0, 0, 3.0, 0, 0]  # Adjust if more s-wave or p-wave is expected

m_Kstar2_val = 1.43
m_Sigma_val = 1.385
m_Nstar_val = 1.71

Gamma_Kstar2_val = 0.62
Gamma_Sigma_val = 0.2
Gamma_Nstar_val = 0.27

l12_val = 2  # I still use 2 assuming K^*_2
l23_val = 1
l31_val = 0

parameters_default = {
    m_Kstar2: m_Kstar2_val,
    m_Sigma: m_Sigma_val,
    m_Nstar: m_Nstar_val,
    Gamma_Kstar2: Gamma_Kstar2_val,
    Gamma_Sigma: Gamma_Sigma_val,
    Gamma_Nstar: Gamma_Nstar_val,
    l12: l12_val,
    l23: l23_val,
    l31: l31_val,
}

a_dict = {a[i]: a_vals[i + l_max] for i in range(-l_max, l_max + 1)}
b_dict = {b[i]: b_vals[i + l_max] for i in range(-l_max, l_max + 1)}
c_dict = {c[i]: c_vals[i + l_max] for i in range(-l_max, l_max + 1)}
parameters_default.update(a_dict)
parameters_default.update(b_dict)
parameters_default.update(c_dict)

Latex(aslatex(parameters_default))